In [ ]:
import osfrom pathlib import Pathfrom datetime import datetime, timedelta, timezonefrom sentinelhub import (    SHConfig, BBox, CRS, MimeType, SentinelHubRequest, SentinelHubSession,    DataCollection, bbox_to_dimensions, SentinelHubCatalog)from PIL import Imageimport geopandas as gpdimport numpy as npfrom shapely.geometry import shape, boxfrom sqlalchemy import create_engine, textfrom src.config.settings import Configfrom src.sentinel.auth import sh_configfrom src.sentinel.catalog_search import search_imagesfrom src.sentinel.download import download_imagefrom src.processing.enhancements import enhance_imagefrom src.processing.bbox import create_valid_bboxfrom src.db.connection import create_db_engineimport matplotlib.pyplot as plt

In [ ]:
def main():    print("🚀 Démarrage de l'exploration des données Sentinel-2")    cfg = sh_config()    catalog = SentinelHubCatalog(config=cfg)    print("✅ Connexion à Sentinel Hub réussie!")    engine = create_db_engine()    samples = get_sample_geometries(engine, Config.MAX_PATCHES)    if not samples:        print("❌ Aucune géométrie trouvée!")        return    gid, geom = samples[0]    print(f"\n🔍 Traitement de la zone {gid}")    bbox = create_valid_bbox(geom, Config.PATCH_SIZE_M)    if not bbox:        return    time_interval = format_time_interval(Config.TIME_INTERVAL)    print(f"🕒 Période formatée: {time_interval}")    images = search_images(catalog, bbox, time_interval, Config.MAX_CLOUD_COVER)    if not images:        print("\n⚠️ Aucune image disponible pour cette zone/date.")        return    for image in images:        output_file = Config.OUTPUT_DIR / f"mangrove_{gid}_{image['properties']['datetime']}.png"        success = download_image(cfg, bbox, time_interval, output_file)        if success:            print(f"✅ Image téléchargée: {output_file}")            enhanced_image = enhance_image(output_file)            plt.imshow(enhanced_image)            plt.title(f'Image améliorée pour {gid}')            plt.show()        else:            print("❌ Échec du téléchargement.")if __name__ == '__main__':    main()